# Setup

In [ ]:
%pip install -e ..[dev]

# 🤖 Focoos Foundational Models


In [ ]:
from focoos import Focoos, FocoosEnvHostUrl
from pprint import pprint
import os

focoos = Focoos(api_key=os.getenv("FOCOOS_API_KEY"), host_url=FocoosEnvHostUrl.DEV)

pprint(focoos.list_focoos_models())

## Cloud Inference

In [ ]:
from pprint import pprint
import os
from focoos import DeploymentMode, Focoos, FocoosEnvHostUrl
from supervision import plot_image


model_ref = "focoos_object365"
image_path = "./assets/ade_val_034.jpg"

focoos = Focoos(api_key=os.getenv("FOCOOS_API_KEY"), host_url=FocoosEnvHostUrl.DEV)

model = focoos.get_model(model_ref)
## Only admin can deploy foundational models
# model.deploy(deployment_mode=DeploymentMode.REMOTE, wait=True)

output, preview = model.remote_infer(image_path, threshold=0.4, annotate=True)
plot_image(preview)

### Unload Model

In [ ]:
## Only admin can deploy foundational models
model.unload()

## Local Inference

This section demonstrates how to perform local inference using a model from the Focoos platform. 
We will load a model, deploy it locally, and then run inference on a sample image.


In [ ]:
from focoos import Focoos, FocoosEnvHostUrl, DeploymentMode
import os
from pprint import pprint
from supervision import plot_image


focoos = Focoos(
    api_key=os.getenv("FOCOOS_API_KEY"),
    host_url=FocoosEnvHostUrl.DEV,
)
image_path = "./assets/ade_val_034.jpg"
model_ref = "focoos_object365"

model_info = focoos.get_model_info(model_ref)

model = focoos.get_model(model_ref)
model.deploy(deployment_mode=DeploymentMode.LOCAL)

# latency = model.benchmark(iterations=10, size=640)
# pprint(latency)
output, preview = model.infer(image_path, threshold=0.3, annotate=True)
pprint(output.detections)
pprint(output.latency)

plot_image(preview)

# User Models

### List User Models

In [ ]:
from pprint import pprint
import os
from focoos import Focoos, FocoosEnvHostUrl
from dotenv import load_dotenv


load_dotenv()

focoos = Focoos(api_key=os.getenv("FOCOOS_API_KEY"), host_url=FocoosEnvHostUrl.DEV)

models = focoos.list_models()
pprint(models)

## Create Model

In [ ]:
from pprint import pprint
import os
from focoos import DeploymentMode, Focoos, FocoosEnvHostUrl
from dotenv import load_dotenv


load_dotenv()

focoos = Focoos(api_key=os.getenv("FOCOOS_API_KEY"), host_url=FocoosEnvHostUrl.LOCAL)

model = focoos.new_model(
    name="test-model", focoos_model="focoos_object365", description="Test model"
)
### Get Model Info

## Deploy user model on shared cloud endpoint

In [ ]:
from pprint import pprint
import os
from focoos import Focoos, FocoosEnvHostUrl
from dotenv import load_dotenv


load_dotenv()

focoos = Focoos(api_key=os.getenv("FOCOOS_API_KEY"), host_url=FocoosEnvHostUrl.DEV)

model = focoos.get_model("fa94df6806c84c11")
model_info = model.get_info()
pprint(model_info.location)
model.deploy(deployment_mode=DeploymentMode.REMOTE, wait=True)

## Cloud Inference

In [ ]:
from pprint import pprint
import os
from focoos import Focoos, FocoosEnvHostUrl
from dotenv import load_dotenv
import cv2
import numpy as np
import supervision as sv

load_dotenv()

focoos = Focoos(api_key=os.getenv("FOCOOS_API_KEY"), host_url=FocoosEnvHostUrl.DEV)

model = focoos.get_model("fa94df6806c84c11")
model_info = model.get_info()
image_path = "./assets/aquarium.jpg"

output, preview = model.remote_infer(image_path, threshold=0.5, annotate=True)
sv.plot_image(preview)

### Model unload 

In [ ]:
model.unload()

## Local Inference




In [ ]:
from focoos import Focoos, FocoosEnvHostUrl, DeploymentMode
import os
from pprint import pprint
from supervision import plot_image


focoos = Focoos(
    api_key=os.getenv("FOCOOS_API_KEY"),
    host_url=FocoosEnvHostUrl.DEV,
)
image_path = "./assets/aquarium.jpg"
model_ref = "focoos_object365"

model_info = focoos.get_model_info(model_ref)

model = focoos.get_model(model_ref)
model.deploy(deployment_mode=DeploymentMode.LOCAL)

latency = model.benchmark(iterations=10, size=640)
# pprint(latency)
output, preview = model.infer(image_path, threshold=0.3, annotate=True)

plot_image(preview)

## Train a Model

### List Public Datasets


In [ ]:
from pprint import pprint
import os
from focoos import Focoos, FocoosEnvHostUrl
from dotenv import load_dotenv


load_dotenv()

focoos = Focoos(api_key=os.getenv("FOCOOS_API_KEY"), host_url=FocoosEnvHostUrl.LOCAL)

models = focoos.list_models()
pprint(models)

model = focoos.new_model(
    name="test-model-2", focoos_model="focoos_object365", description="Test model"
)


model = focoos.get_model("fa94df6806c84c11")

datasets = focoos.list_shared_datasets()
pprint(datasets)

In [ ]:
from focoos.ports import Hyperparameters, TrainInstance


res = model.remote_train(
    anyma_version="0.11.1",
    dataset_ref="11e80dd77806450f",
    instance_type=TrainInstance.ML_G4DN_XLARGE,
    volume_size=50,
    max_runtime_in_seconds=36000,
    hyperparameters=Hyperparameters(
        learning_rate=0.0001,
        batch_size=16,
        max_iters=1500,
        eval_period=100,
        resolution=640,
    ),  # type: ignore
)
pprint(res)

In [ ]:
completed_status = ["Completed", "Failed"]
import time
from focoos.utils.logger import get_logger

logger = get_logger(__name__)

status = model.train_status()
while status["main_status"] not in completed_status:
    status = model.train_status()
    logger.info(f"Training status: {status['main_status']}")
    pprint(f"Training progress: {status['status_transitions']}")
    time.sleep(30)

In [ ]:
logs = model.train_logs()
pprint(logs)